# An Application of the Dynamic Memory Model

The following notebook supplies an application of the Dynamic Memory Model to enhance the capabilities of [dialogue systems](https://en.wikipedia.org/wiki/Dialog_system). This software is intended to test the ability of this model to enhance a dialogue system's ability to understand user input by forming high-level concepts that evolve over time.

The code below is written in Python, and uses a [Neo4j Graph Database](https://neo4j.com/product/) to provide non-volatile storage and efficient querying capabilities.

The training corpus is supplied by the [bAbI Tasks Data 1-20 (v1.2)](https://research.fb.com/downloads/babi/). A sequence of English sentences are provided to provide the system knowledge of a simple domain involving characters moving to different rooms.

### TODO
- Py2neo visualization in Jupyter notebooks http://nicolewhite.github.io/neo4j-jupyter/hello-world.html

### Prerequisites to Running this Notebook
- Python (3.5+)
- Python packages (install via pip): `pandas`, `numpy`, `nltk`, `neo4j-driver`
- [Neo4j](https://neo4j.com/download/) (3.1+): 

## Process the Text

### Import DataFrames
First we will use `pandas` to import `qa1_single-supporting-fact_train.txt` from our corpus into a DataFrame. The sentences will be tokenized using `nltk`.

In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
# Read the bAbI data as CSV
filename = 'src/main/resources/qa1_single-supporting-fact_train.txt'
data = pd.read_csv(filename, delimiter='\t', names=['sentence', 'answer', 'factid'])
data = data.fillna('')

# Use NLTK to tokenize the sentences into arrays of words
tokenize = lambda row: nltk.word_tokenize(row['sentence'])[1:]
data['sentence'] = data.apply(tokenize, axis=1)

# Create a DataFrame with just the statements
statements = data[data['answer'] == ''] \
    .reset_index(drop=True) \
    .drop('answer', axis=1) \
    .drop('factid', axis=1)

# Create a DataFrame with just the questions
questions = data[data['answer'] != ''] \
    .reset_index(drop=True)

The `data` DataFrame contains all lines of text, which can be either statements, or questions about the preceeding statements. This is further split up into the `statements` and `questions` DataFrames for easy access to all statements and questions respectively.

In [3]:
data[:6]

,sentence,answer,factid
0,"[Mary, moved, to, the, bathroom, .]",,
1,"[John, went, to, the, hallway, .]",,
2,"[Where, is, Mary, ?]",bathroom,1
3,"[Daniel, went, back, to, the, hallway, .]",,
4,"[Sandra, moved, to, the, garden, .]",,
5,"[Where, is, Daniel, ?]",hallway,4


In [4]:
statements[:4]

,sentence
0,"[Mary, moved, to, the, bathroom, .]"
1,"[John, went, to, the, hallway, .]"
2,"[Daniel, went, back, to, the, hallway, .]"
3,"[Sandra, moved, to, the, garden, .]"


In [5]:
questions[:2]

,sentence,answer,factid
0,"[Where, is, Mary, ?]",bathroom,1
1,"[Where, is, Daniel, ?]",hallway,4


### Entity Extraction
Next, we will extract the relevant entities from each statement. Due to the simplicity of the data, each statement can be thought of as a `(subject, relation, object)` triple. We would like to define a function `E`, that when given a sequence of tokens, produces this triple. For instance,
```
E([Mary, moved, to, the, bathroom]) = (Mary, moved, bathroom).
```
This allows one to construct a graph of relationships between objects, as we will see in the next sections.

The training data is simple enough that we can tag each word with its part of speech to find the triple directly. 

In [6]:
# Tag each token as a part of speech
pos_tag = lambda row: nltk.pos_tag(row['sentence'])
statements['tag'] = statements.apply(pos_tag, axis=1)

In [7]:
def extract_triple(tags):
    '''Extracts a (subject, relation, object) triple from each sentence based on the POS tags'''
    subject, relation, obj = '', '', ''
    for word,tag in tags:
        if tag == 'NNP':
            subject = word
        elif tag == 'VBD':
            relation = word
        elif tag == 'NN':
            obj = word
    return (subject, relation, obj)

In [8]:
statements['triple'] = statements.apply(lambda row: extract_triple(row['tag']), axis=1)

In [9]:
statements[:5]

,sentence,tag,triple
0,"[Mary, moved, to, the, bathroom, .]","[(Mary, NNP), (moved, VBD), (to, TO), (the, DT...","(Mary, moved, bathroom)"
1,"[John, went, to, the, hallway, .]","[(John, NNP), (went, VBD), (to, TO), (the, DT)...","(John, went, hallway)"
2,"[Daniel, went, back, to, the, hallway, .]","[(Daniel, NNP), (went, VBD), (back, RB), (to, ...","(Daniel, went, hallway)"
3,"[Sandra, moved, to, the, garden, .]","[(Sandra, NNP), (moved, VBD), (to, TO), (the, ...","(Sandra, moved, garden)"
4,"[John, moved, to, the, office, .]","[(John, NNP), (moved, VBD), (to, TO), (the, DT...","(John, moved, office)"


### Debug Functions

These are handy debugging functions that we will use later.

In [10]:
def person_data(person):
    '''Get all statements that refer to the specified person'''
    return statements[statements['triple'].map(lambda t: t[0] == person)]

In [11]:
person_data('Sandra')[:3]

,sentence,tag,triple
3,"[Sandra, moved, to, the, garden, .]","[(Sandra, NNP), (moved, VBD), (to, TO), (the, ...","(Sandra, moved, garden)"
5,"[Sandra, journeyed, to, the, bathroom, .]","[(Sandra, NNP), (journeyed, VBD), (to, TO), (t...","(Sandra, journeyed, bathroom)"
10,"[Sandra, travelled, to, the, office, .]","[(Sandra, NNP), (travelled, VBD), (to, TO), (t...","(Sandra, travelled, office)"


In [12]:
def most_recent(person, n=5):
    '''Get the n most recent statements that refer to the specified person in reverse chronological order'''
    return person_data(person)[-n:].iloc[::-1]

In [13]:
most_recent('Daniel', n=3)

,sentence,tag,triple
1999,"[Daniel, went, to, the, garden, .]","[(Daniel, NNP), (went, VBD), (to, TO), (the, D...","(Daniel, went, garden)"
1996,"[Daniel, travelled, to, the, kitchen, .]","[(Daniel, NNP), (travelled, VBD), (to, TO), (t...","(Daniel, travelled, kitchen)"
1992,"[Daniel, moved, to, the, office, .]","[(Daniel, NNP), (moved, VBD), (to, TO), (the, ...","(Daniel, moved, office)"


## Build the Graph
Once we have processed the data into triples, we can build graphs from them. Below we have defined a couple functions to reset the database and run queries.

In [14]:
from neo4j.v1 import GraphDatabase, basic_auth

In [15]:
# Create a neo4j session
driver = GraphDatabase.driver('bolt://localhost:7687', auth=basic_auth('neo4j', 'neo4j'))

In [16]:
# WARNING: This will clear the database when run!
def reset_db():
    session = driver.session()
    session.run('MATCH (n) DETACH DELETE n')

In [17]:
# Create a graph based on each triple
def create(query):
    session = driver.session()
    for subject,relation,obj in statements['triple']:
        session.run(query, { 
            'subject': subject,
            'relation': relation,
            'obj': obj
        })

### V1: Direct relationships
One of the first impulses when building the graph is to represent the subject and object as nodes, and the relations as edges between them.

In [18]:
reset_db()

In [19]:
# Create a direct relationship between subject and object
create('''
    MERGE (s:SUBJECT {name: $subject}) 
    MERGE (o:OBJECT  {name: $obj}) 
    MERGE (s)-[r:RELATION {name: $relation}]->(o)
''')

Run the query below and see what the graph looks like. Pop open a new tab in the Neo4j browser (default http://localhost:7474/browser/) and run the query:
```
MATCH (n) RETURN n LIMIT 50
```
The graph is a reasonable first start, as the relations point each person to where they have been. But this poses a potential problem: how do we know where each person is right now, or where they have been previously? All we can know from the graph is how many times a person was in each room because they have visited them all multiple times.

### V2: Nodes for relationships
One approach is to form a linked list of "events". Each event corresponds to a person updating the room that they are in. Since we chose edges to be our relations, we cannot form edges between relations. To alleviate this, we can transform the relation to a node, and draw two edges to form a 3-node triple.

In [20]:
reset_db()

In [21]:
# Represent each relation as a node
create('''
    MERGE (s:SUBJECT {name: $subject})
    MERGE (o:OBJECT  {name: $obj})
    CREATE (s)-[:R0]->(r:RELATION {name: $relation})-[:R1]->(o)
''')

Run the query again and see what changed

### V3: Linked list of relationships
The final step is to build the linked list based on the order in which the relations were created. This will allow us to not only find the room a person is in right now, but produce a list of rooms that they were in, in the order that they were visited.

In [22]:
reset_db()

In [23]:
# Represent each relation as a node, ordered by a linked list (per subject)
create('''
    MERGE (s:SUBJECT {name: $subject})
    MERGE (o:OBJECT  {name: $obj})
    CREATE (s)-[:R0]->(r:RELATION {name: $relation})-[:R1]->(o)

    WITH s,r,o

    MATCH (s)-[:R0]->(r2:RELATION)
    WHERE r2 <> r AND NOT (r2)-[:NEXT]->() 
    CREATE (r2)-[:NEXT]->(r)
''')

Check the new graph out and see what changed. It's helpful to change the colors of the nodes and edges to visualize this better.

## Query the Graph
Now we can ask the graph useful questions.

In [24]:
def find_person(person):
    '''Find the room a person is currently in'''
    query = '''
        MATCH (s:SUBJECT {name:$name})-->(r:RELATION)-->(o:OBJECT)
        WHERE NOT (r)-[:NEXT]->()
        RETURN s,r,o
    '''
    session = driver.session()
    return session.run(query, {'name': person})

Where is Mary?

In [25]:
record = find_person('Mary').single()
print(record['o'].get('name'))

kitchen


Verify that this is true

In [26]:
most_recent('Mary', n=1)

,sentence,tag,triple
1994,"[Mary, journeyed, to, the, kitchen, .]","[(Mary, NNP), (journeyed, VBD), (to, TO), (the...","(Mary, journeyed, kitchen)"


In [27]:
def find_person_history(person, n=100):
    '''Find the list of rooms a person was in, ordered by recency'''
    length = str(n) if n >= 1 else ''
    
    query = '''
        MATCH (s:SUBJECT {name:$name})-->(r:RELATION)-->(o:OBJECT)
        WHERE NOT (r)-[:NEXT]->()
        
        WITH s,r,o
        
        MATCH (s)-->(r_prev:RELATION)-[k*1..%s]->(r), (r_prev)-->(o_prev:OBJECT)
        
        WITH size(k) AS dist, r, o, r_prev, o_prev
        ORDER BY size(k)
        
        WITH r, o, r_prev, o_prev
        RETURN [r.name] + collect(r_prev.name) AS relation, [o.name] + collect(o_prev.name) AS obj
    '''
    query = query % length
    
    session = driver.session()
    record = session.run(query, {'name': person}).single()
    history = list(zip(record['relation'], record['obj']))[:-1]
    
    return history

Where has John been recently?

In [28]:
find_person_history('John', n=5)

[('went', 'bedroom'),
 ('went', 'garden'),
 ('went', 'office'),
 ('', 'bedroom'),
 ('travelled', 'hallway')]

Verify that John has been to to those places, in that order

In [29]:
most_recent('John', n=5)

,sentence,tag,triple
1995,"[John, went, back, to, the, bedroom, .]","[(John, NNP), (went, VBD), (back, RB), (to, TO...","(John, went, bedroom)"
1989,"[John, went, back, to, the, garden, .]","[(John, NNP), (went, VBD), (back, RB), (to, TO...","(John, went, garden)"
1986,"[John, went, back, to, the, office, .]","[(John, NNP), (went, VBD), (back, RB), (to, TO...","(John, went, office)"
1982,"[John, journeyed, to, the, bedroom, .]","[(John, NNP), (journeyed, NN), (to, TO), (the,...","(John, , bedroom)"
1979,"[John, travelled, to, the, hallway, .]","[(John, NNP), (travelled, VBD), (to, TO), (the...","(John, travelled, hallway)"


Note that some of the triples may be blank. This is because the POS tagger made a mistake when tagging the tokens.